In [15]:
pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/80/6f/57d36f6507e432d7fc1956b2e9e8530c5c2d2bfcd8821bcbfae271cd6688/tensorflow-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.14.0 from https://files.pythonhosted.org/packages/ad/6e/1bfe367855dd87467564f7bf9fa14f3b17889988e79598bc37bf18f5ffb6/tensorflow_intel-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv', index_col = 'RowNumber')
dataset.head()


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#Customer ID and Surname would not be relevant as features
X_columns = dataset.columns.tolist()[2:12]
Y_columns = dataset.columns.tolist()[-1:]
print(X_columns)
print(Y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [5]:
X = dataset[X_columns].values 
Y = dataset[Y_columns].values

In [6]:
#We need to encode categorical variables such as geography and gender
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])

In [7]:
#Lets Encode gender now
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

We are treating countries with ordinal values(0 < 1 < 2) but they are incomparable.
To solve this we can use one hot encoding.
We will perform some standardization 

In [8]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ], 
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [9]:
#Standardize the features
X = pipeline.fit_transform(X)

In [10]:
#Spilt the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [16]:
#Let us create the Neural Network
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [17]:
#Initialize ANN
classifier = Sequential()

In [18]:
#Add input layer and hidden layer
classifier.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
classifier.add(Dropout(rate = 0.1))

In [19]:
#Add second layer
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [20]:
#Add output layer
classifier.add(Dense(1, activation = 'sigmoid'))

In [21]:
#Let us take a look at our network
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dropout (Dropout)           (None, 6)                 0         
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dropout_1 (Dropout)         (None, 6)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121 (484.00 Byte)
Trainable params: 121 (484.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
#Optimize the weights
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [23]:
#Fitting the Neural Network
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split = 0.1, verbose = 2)

Epoch 1/200
225/225 - 2s - loss: 0.5792 - accuracy: 0.7900 - val_loss: 0.5191 - val_accuracy: 0.7950 - 2s/epoch - 7ms/step
Epoch 2/200
225/225 - 0s - loss: 0.5098 - accuracy: 0.7957 - val_loss: 0.4648 - val_accuracy: 0.7950 - 482ms/epoch - 2ms/step
Epoch 3/200
225/225 - 1s - loss: 0.4723 - accuracy: 0.7964 - val_loss: 0.4384 - val_accuracy: 0.7975 - 501ms/epoch - 2ms/step
Epoch 4/200
225/225 - 0s - loss: 0.4634 - accuracy: 0.7992 - val_loss: 0.4309 - val_accuracy: 0.8025 - 416ms/epoch - 2ms/step
Epoch 5/200
225/225 - 0s - loss: 0.4597 - accuracy: 0.8022 - val_loss: 0.4272 - val_accuracy: 0.8000 - 495ms/epoch - 2ms/step
Epoch 6/200
225/225 - 0s - loss: 0.4499 - accuracy: 0.8039 - val_loss: 0.4234 - val_accuracy: 0.8125 - 489ms/epoch - 2ms/step
Epoch 7/200
225/225 - 0s - loss: 0.4471 - accuracy: 0.8126 - val_loss: 0.4205 - val_accuracy: 0.8112 - 492ms/epoch - 2ms/step
Epoch 8/200
225/225 - 0s - loss: 0.4433 - accuracy: 0.8062 - val_loss: 0.4175 - val_accuracy: 0.8125 - 420ms/epoch - 2ms/

Epoch 66/200
225/225 - 1s - loss: 0.3677 - accuracy: 0.8421 - val_loss: 0.3264 - val_accuracy: 0.8625 - 503ms/epoch - 2ms/step
Epoch 67/200
225/225 - 0s - loss: 0.3644 - accuracy: 0.8436 - val_loss: 0.3276 - val_accuracy: 0.8650 - 484ms/epoch - 2ms/step
Epoch 68/200
225/225 - 0s - loss: 0.3655 - accuracy: 0.8457 - val_loss: 0.3259 - val_accuracy: 0.8637 - 479ms/epoch - 2ms/step
Epoch 69/200
225/225 - 1s - loss: 0.3658 - accuracy: 0.8465 - val_loss: 0.3254 - val_accuracy: 0.8650 - 543ms/epoch - 2ms/step
Epoch 70/200
225/225 - 1s - loss: 0.3659 - accuracy: 0.8442 - val_loss: 0.3240 - val_accuracy: 0.8637 - 538ms/epoch - 2ms/step
Epoch 71/200
225/225 - 0s - loss: 0.3630 - accuracy: 0.8438 - val_loss: 0.3245 - val_accuracy: 0.8675 - 494ms/epoch - 2ms/step
Epoch 72/200
225/225 - 0s - loss: 0.3616 - accuracy: 0.8474 - val_loss: 0.3232 - val_accuracy: 0.8650 - 477ms/epoch - 2ms/step
Epoch 73/200
225/225 - 1s - loss: 0.3596 - accuracy: 0.8474 - val_loss: 0.3230 - val_accuracy: 0.8662 - 612ms/e

Epoch 131/200
225/225 - 0s - loss: 0.3618 - accuracy: 0.8493 - val_loss: 0.3214 - val_accuracy: 0.8725 - 490ms/epoch - 2ms/step
Epoch 132/200
225/225 - 1s - loss: 0.3585 - accuracy: 0.8517 - val_loss: 0.3210 - val_accuracy: 0.8687 - 503ms/epoch - 2ms/step
Epoch 133/200
225/225 - 0s - loss: 0.3582 - accuracy: 0.8533 - val_loss: 0.3213 - val_accuracy: 0.8675 - 434ms/epoch - 2ms/step
Epoch 134/200
225/225 - 0s - loss: 0.3600 - accuracy: 0.8475 - val_loss: 0.3207 - val_accuracy: 0.8662 - 499ms/epoch - 2ms/step
Epoch 135/200
225/225 - 1s - loss: 0.3535 - accuracy: 0.8533 - val_loss: 0.3203 - val_accuracy: 0.8700 - 518ms/epoch - 2ms/step
Epoch 136/200
225/225 - 1s - loss: 0.3571 - accuracy: 0.8511 - val_loss: 0.3195 - val_accuracy: 0.8700 - 504ms/epoch - 2ms/step
Epoch 137/200
225/225 - 0s - loss: 0.3555 - accuracy: 0.8519 - val_loss: 0.3205 - val_accuracy: 0.8725 - 459ms/epoch - 2ms/step
Epoch 138/200
225/225 - 0s - loss: 0.3580 - accuracy: 0.8532 - val_loss: 0.3199 - val_accuracy: 0.8712 -

225/225 - 1s - loss: 0.3600 - accuracy: 0.8526 - val_loss: 0.3206 - val_accuracy: 0.8725 - 858ms/epoch - 4ms/step
Epoch 196/200
225/225 - 1s - loss: 0.3611 - accuracy: 0.8514 - val_loss: 0.3203 - val_accuracy: 0.8763 - 635ms/epoch - 3ms/step
Epoch 197/200
225/225 - 1s - loss: 0.3556 - accuracy: 0.8521 - val_loss: 0.3197 - val_accuracy: 0.8737 - 649ms/epoch - 3ms/step
Epoch 198/200
225/225 - 1s - loss: 0.3606 - accuracy: 0.8522 - val_loss: 0.3200 - val_accuracy: 0.8737 - 694ms/epoch - 3ms/step
Epoch 199/200
225/225 - 1s - loss: 0.3639 - accuracy: 0.8468 - val_loss: 0.3209 - val_accuracy: 0.8725 - 543ms/epoch - 2ms/step
Epoch 200/200
225/225 - 1s - loss: 0.3580 - accuracy: 0.8518 - val_loss: 0.3206 - val_accuracy: 0.8725 - 523ms/epoch - 2ms/step


In [24]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 [==============================] - 0s 2ms/step
[[0.3027615 ]
 [0.22279063]
 [0.16503797]
 [0.0668716 ]
 [0.11304154]]


In [25]:
#Let us use confusion matrix with cutoff value as 0.5
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [26]:
#Making the Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1523   72]
 [ 204  201]]


In [27]:
#Accuracy of our NN
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

86.2 % of data was classified correctly
